In [1]:
import qiskit

from qiskit import IBMQ, QuantumCircuit, execute, Aer
from qiskit.quantum_info.analysis.data import marginal_counts
from qiskit.quantum_info.analysis.readout_error_mitigation import backend_readout_error_mitigation

In [2]:
provider = IBMQ.load_account()

/Users/aa406165/Git/qiskit-core/.venv/lib/python3.8/site-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


In [3]:
backend = provider.get_backend('ibmq_vigo')

In [4]:
qc = QuantumCircuit(5)
qc.h(range(5))
qc.measure_all()
job = execute(qc, Aer.get_backend('qasm_simulator'), shots=8192)
result = job.result()
result.get_counts()

{'00000': 260,
 '00001': 286,
 '10000': 271,
 '10001': 251,
 '10010': 227,
 '10011': 238,
 '10100': 238,
 '10101': 240,
 '10110': 268,
 '10111': 272,
 '11000': 254,
 '11001': 254,
 '11010': 243,
 '11011': 267,
 '11100': 245,
 '11101': 272,
 '11110': 255,
 '11111': 251,
 '00010': 279,
 '00011': 234,
 '00100': 262,
 '00101': 231,
 '00110': 258,
 '00111': 255,
 '01000': 253,
 '01001': 271,
 '01010': 260,
 '01011': 273,
 '01100': 242,
 '01101': 252,
 '01110': 279,
 '01111': 251}

In [5]:
qc = QuantumCircuit(5)
qc.h(range(5))
qc.measure_all()
job = execute(qc, backend, shots=8192)
result = job.result()
result.get_counts()

{'00000': 255,
 '00001': 285,
 '10000': 283,
 '10001': 279,
 '10010': 256,
 '10011': 220,
 '10100': 268,
 '10101': 230,
 '10110': 226,
 '10111': 252,
 '11000': 268,
 '11001': 268,
 '11010': 242,
 '11011': 223,
 '11100': 268,
 '11101': 207,
 '11110': 242,
 '11111': 200,
 '00010': 250,
 '00011': 235,
 '00100': 259,
 '00101': 278,
 '00110': 246,
 '00111': 242,
 '01000': 320,
 '01001': 287,
 '01010': 260,
 '01011': 266,
 '01100': 288,
 '01101': 289,
 '01110': 264,
 '01111': 236}

In [6]:
backend_readout_error_mitigation(result, backend)

{'00000': 252.89675960775628,
 '00001': 287.63323087957303,
 '00010': 249.75896876332158,
 '00011': 232.25378550089837,
 '00100': 258.6155613505997,
 '00101': 280.35260504320456,
 '00110': 245.50117240022206,
 '00111': 240.40562334101247,
 '01000': 325.7041537534086,
 '01001': 287.80582336147853,
 '01010': 257.76630167669936,
 '01011': 267.0708308885369,
 '01100': 289.51448630568865,
 '01101': 293.1485977791375,
 '01110': 264.1207399309406,
 '01111': 233.57584074532534,
 '10000': 285.2241679053143,
 '10001': 282.24678471215094,
 '10010': 256.0236468780302,
 '10011': 216.30372286041785,
 '10100': 270.2348143454711,
 '10101': 227.48996677607025,
 '10110': 222.7855553613619,
 '10111': 255.00960101056626,
 '11000': 267.7390171921506,
 '11001': 270.1456832986706,
 '11010': 240.50198924172142,
 '11011': 220.40874003725187,
 '11100': 269.3701231643857,
 '11101': 203.84351736659198,
 '11110': 241.26295028619631,
 '11111': 197.28523823585206}

## Readout Error

In [7]:
n_qubits = backend.configuration().n_qubits
qc0 = QuantumCircuit(n_qubits)
qc0.measure_all()
qc1 = QuantumCircuit(n_qubits)
qc1.x(range(n_qubits))
qc1.measure_all()
job = execute([qc0, qc1], backend, shots=8192)
result = job.result()
result.get_counts()

[{'00000': 7939,
  '00001': 65,
  '10000': 37,
  '10100': 2,
  '11000': 9,
  '00010': 52,
  '00011': 6,
  '00100': 36,
  '00101': 1,
  '01000': 45},
 {'10011': 7,
  '10101': 7,
  '10110': 11,
  '10111': 171,
  '11000': 1,
  '11001': 17,
  '11010': 2,
  '11011': 200,
  '11100': 15,
  '11101': 438,
  '11110': 146,
  '11111': 6971,
  '00101': 1,
  '00111': 18,
  '01001': 1,
  '01011': 2,
  '01101': 10,
  '01110': 2,
  '01111': 172}]

In [8]:
er01 = []
for i in range(n_qubits):
    marginal = marginal_counts(result.get_counts()[0], [i])
    er01.append(marginal['1']/sum(marginal.values()))
er10 = []
for i in range(n_qubits):
    marginal = marginal_counts(result.get_counts()[1], [i])
    er10.append(marginal['0']/sum(marginal.values()))

In [9]:
[(a+b)/2 for a, b in zip(er01, er10)]

[0.01519775390625,
 0.033447265625,
 0.01641845703125,
 0.01641845703125,
 0.0155029296875]

In [10]:
[backend.properties().readout_error(i) for i in range(5)]

[0.010000000000000009,
 0.038999999999999924,
 0.01200000000000001,
 0.019000000000000017,
 0.018000000000000016]